In [1]:
import os
import pickle
import pandas as pd
import multiprocessing as mp
from multiprocessing import Pool
from tqdm import tqdm
import re
import numpy as np
import datetime as dt
from datetime import timezone
from datetime import timedelta
import math
import _thread
import copy

import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [2]:
import config

{'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'time': {'end': 'Mar 13 00:00:00 -0500 2018',
                   'start': 'Mar 12 00:00:00 -0500 2018'}},
 'path': {'cwd': '/Users/lzhou/git/github/uclresearchanalysis/data/givenchy',
          'friends_dictionary': '/Users/lzhou/git/github/uclresearchanalysis/other/friends_dictionary.dat',
          'pickle': '/Users/lzhou/git/github/uclresearchanalysis/data/givenchy/pickle',
          'processed_dataframe': '/Users/lzhou/git/github/uclresearchanalysis/data/givenchy/pickle/final_processed_data.dat',
          'result': '/Users/lzhou/git/github/uclresearchanalysis/data/givenchy/result',
          'twitter': '/Users/lzhou/git/github/uclresearchanalysis/data/givenchy/twitter'}}


In [3]:
from config import load_friends_dictionary
friends_dictionary = load_friends_dictionary()
pickle_path = config.settings['path']['pickle']

'Loaded 21533 friends dictionary'


In [4]:
def find_by_user_name(df, user_name):
    try:
        user = df[df.user == user_name]
        return user.user_id.iloc[0]
    except IndexError:
        return ''

def find_by_user_id(df, user_id):
    try:
        user = df[df.user_id == user_id]
        return user.user.iloc[0]
    except IndexError:
        return ''
    
def find_nearest_friend_in_past(df, index, unchecked_friend_ids):
    try:
        user = df[df.user_id.isin(unchecked_friend_ids)]
        return user.user_id.iloc[-1]
    except IndexError:
        return '' 

def find_source(df, index, friends_dictionary, user_id, affected_users):
    try:
        if df.loc[index, 'source_type'] == 'RT' or df.loc[index, 'source_type'] == 'AT' :
            df.loc[index, 'source_user_id'] = find_by_user_name(df, df.loc[index, 'source_user'])
        if df.loc[index, 'source_type'] == 'RP':
            df.loc[index, 'source_user'] = find_by_user_id(df, df.loc[index, 'source_user_id'])
        if df.loc[index, 'source_type'] == '':
            friends = set(friends_dictionary[user_id])
            unchecked_friend_ids = friends - affected_users
            nearest_friend_id = find_nearest_friend_in_past(df, index, unchecked_friend_ids)
            if nearest_friend_id != '':
                df.loc[index, 'source_type'] = 'FR'
                df.loc[index, 'source_user_id'] = nearest_friend_id
                df.loc[index, 'source_user'] = find_by_user_id(df, nearest_friend_id)
            else:
                df.loc[index, 'source_type'] = 'OR'
    except KeyError as e:
        print(e)

In [5]:
with open (pickle_path + '/user_unique.dat', 'rb') as all_unique_tweets_file:
    df = pickle.load(all_unique_tweets_file)
print('Loaded {} tweets'.format(len(df)))
affected_users = set([])
index = len(df) - 1
with tqdm(total = len(df), unit='Tweet', unit_scale=True, unit_divisor=1024) as pbar:
    while index >= 0:
        user_id = df.loc[index, 'user_id']
        find_source(df, index, friends_dictionary, user_id, affected_users)
        affected_users.add(user_id)
        index -= 1
        if (index % 100) == 0:
            with open(pickle_path + '/user_unique_after_lookup.dat', 'wb') as tweet_data_frame_after_source_lookup:
                pickle.dump(df, tweet_data_frame_after_source_lookup)
        pbar.update()
with open(pickle_path + '/user_unique_after_lookup.dat', 'wb') as tweet_data_frame_after_source_lookup:
    pickle.dump(df, tweet_data_frame_after_source_lookup)

  0%|          | 0.00/4.64k [00:00<?, ?Tweet/s]

Loaded 4754 tweets


100%|██████████| 4.64k/4.64k [00:16<00:00, 292Tweet/s]


In [6]:
for index in tqdm(range(len(df))):
    type = df.loc[index, 'source_type']
    if (type == 'OR'):
        df.loc[index, 'seed_user'] = df.loc[index, 'user']
        df.loc[index, 'seed_user_id'] = df.loc[index, 'user_id']
        df.loc[index, 'generation'] = 1
    else:
        try:
            source = df[df.user_id == df.loc[index, 'source_user_id']].iloc[0]
            df.loc[index, 'seed_user'] = source.user
            df.loc[index, 'seed_user_id'] = source.user_id
            df.loc[index, 'generation'] = source.generation + 1    
        except:
            df.loc[index, 'seed_user'] = 'unknown'
            df.loc[index, 'seed_user_id'] = 'unknown'
            df.loc[index, 'generation'] = 'unknown'

100%|██████████| 4754/4754 [00:22<00:00, 214.99it/s]


In [15]:
df['minutes'] = 0
first_tweet_datetime = df.created_at.iloc[0]
for index in tqdm(range(len(df))):
    df.loc[index, 'minutes'] = round((df.loc[index, 'created_at'] - first_tweet_datetime).total_seconds() / 60.0, 2)

100%|██████████| 4754/4754 [00:05<00:00, 912.51it/s]


In [16]:
with open(config.settings['path']['processed_dataframe'], 'wb') as tweet_data_frame_after_source_lookup:
    pickle.dump(df, tweet_data_frame_after_source_lookup)

In [17]:
df.head()

,user,user_id,created_at,source_type,source_user,source_user_id,seed_user,seed_user_id,followers_count,friends_count,text,location,country_code,long,latt,state,mentions,generation,minutes
0,davelackie,100766356,2018-03-12 08:20:58-05:00,OR,,,davelackie,100766356,143119,4643,So sad to hear that fashion designer Hubert de...,Canada,,NaN,NaN,,[],1,0.00
1,alexanderskhan,3186545203,2018-03-12 08:21:07-05:00,RT,davelackie,100766356,davelackie,100766356,753,3428,RT @davelackie: So sad to hear that fashion de...,"New Orleans, LA",,NaN,NaN,Louisiana,[davelackie],2,0.15
2,consiglierela,4134992843,2018-03-12 08:21:12-05:00,RT,davelackie,100766356,davelackie,100766356,2161,396,RT @davelackie: So sad to hear that fashion de...,⭐,,NaN,NaN,,[davelackie],2,0.23
3,ImpactPrincess,1955472014,2018-03-12 08:21:36-05:00,RT,davelackie,100766356,davelackie,100766356,242,642,RT @davelackie: So sad to hear that fashion de...,"David Jones, Probably",,NaN,NaN,,[davelackie],2,0.63
4,Damek0Masca,406301175,2018-03-12 08:22:06-05:00,OR,,,Damek0Masca,406301175,116,239,Today we mourn the death of a #fashion legend;...,New York,,NaN,NaN,New York,[],1,1.13
